In [22]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp0tgfxklb/pubring.gpg' created
gpg: /tmp/tmp0tgfxklb/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip install -q keras

In [4]:
!python3 drive/Colab/mnist_cnn.py

Using TensorFlow backend.
11493376/11490434 [==============================] - 2s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
2018-07-17 23:35:18.337521: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:897] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-07-17 23:35:18.338104: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-07-17 23:35:18.338150: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-17 23:35:18.841195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-17 23:35:18.841370: I tensorflow/core

60000/60000 [==============================] - 14s 232us/step - loss: 0.2589 - acc: 0.9218 - val_loss: 0.0600 - val_acc: 0.9820
Epoch 2/12
60000/60000 [==============================] - 12s 192us/step - loss: 0.0880 - acc: 0.9738 - val_loss: 0.0381 - val_acc: 0.9870
Epoch 3/12
51456/60000 [========================>.....] - ETA: 1s - loss: 0.0666 - acc: 0.9803

60000/60000 [==============================] - 11s 187us/step - loss: 0.0660 - acc: 0.9804 - val_loss: 0.0354 - val_acc: 0.9885
Epoch 4/12
60000/60000 [==============================] - 11s 187us/step - loss: 0.0523 - acc: 0.9836 - val_loss: 0.0323 - val_acc: 0.9891
Epoch 5/12
59008/60000 [============================>.] - ETA: 0s - loss: 0.0459 - acc: 0.9860

60000/60000 [==============================] - 11s 189us/step - loss: 0.0462 - acc: 0.9860 - val_loss: 0.0283 - val_acc: 0.9905
Epoch 6/12
60000/60000 [==============================] - 11s 190us/step - loss: 0.0414 - acc: 0.9876 - val_loss: 0.0280 - val_acc: 0.9910
Epoch 7/12
60000/60000 [==============================] - 11s 188us/step - loss: 0.0375 - acc: 0.9886 - val_loss: 0.0290 - val_acc: 0.9906
Epoch 8/12
  384/60000 [..............................] - ETA: 14s - loss: 0.0389 - acc: 0.9870

60000/60000 [==============================] - 11s 190us/step - loss: 0.0339 - acc: 0.9891 - val_loss: 0.0281 - val_acc: 0.9915
Epoch 9/12
60000/60000 [==============================] - 12s 197us/step - loss: 0.0319 - acc: 0.9904 - val_loss: 0.0259 - val_acc: 0.9918
Epoch 10/12
43392/60000 [====================>.........] - ETA: 3s - loss: 0.0289 - acc: 0.9914

60000/60000 [==============================] - 11s 191us/step - loss: 0.0292 - acc: 0.9912 - val_loss: 0.0341 - val_acc: 0.9892
Epoch 11/12
60000/60000 [==============================] - 11s 191us/step - loss: 0.0281 - acc: 0.9911 - val_loss: 0.0238 - val_acc: 0.9927
Epoch 12/12
55552/60000 [==========================>...] - ETA: 0s - loss: 0.0261 - acc: 0.9918

60000/60000 [==============================] - 11s 191us/step - loss: 0.0266 - acc: 0.9918 - val_loss: 0.0267 - val_acc: 0.9917
Test loss: 0.02671625117767453
Test accuracy: 0.9917


In [0]:
!ls -l drive1

In [28]:
# style transfer
from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg16
from keras import backend as K

base_image_path = "drive/Colab/images/input.jpg"
style_reference_image_path = "drive/Colab/images/startnight.jpg"
result_prefix = "output"
iterations = 2 # 10

# 原圖與風格圖佔output比重
content_weight = 0.025
style_weight = 1.0
# 損失總差異預設值
total_variation_weight = 1.0

# output 圖的寬高
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 轉換成 VGG 16 input 格式
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

# 將特徵向量轉換成 image
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    # 設定RGB顏色的中心點 (Remove zero-center by mean pixel)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# 設定 Keras 變數 base_image = 原圖 向量
base_image = K.variable(preprocess_image(base_image_path))
# 設定 Keras 變數 style_reference_image = 風格圖 向量
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

# 設定合成圖的起始值
if K.image_data_format() == 'channels_first':
    combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
else:
    combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

# 合併原圖、風格圖、合成圖 向量
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

# 載入 VGG 16 模型，不包括加在最後3層的卷積層
model = vgg16.VGG16(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)

# print model information                    
print(model.summary())

# save model pictures to VGG16.png                    
from keras.utils import plot_model
#plot_model(model, to_file='./VGG16.png')

                    
# get the symbolic outputs of each "key" layer (we gave them unique names).
# 讀取 VGG 16 模型的每一層的名稱與output
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# 計算 風格 loss 的 gram matrix
def gram_matrix(x):
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 計算 風格 loss 
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

# 計算 content loss 
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent


# 計算 損失總差異(total variation loss)，以利合成圖的連貫性
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

# 彙總上面三項的損失(loss)
loss = K.variable(0.)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

# 計算合成圖的梯度(gradients)
grads = K.gradients(loss, combination_image)

# 建立 Keras function API 模型
outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)


# 依梯度下降法，評估模型
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# 評估模型類別
# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 執行模型評估
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

# 在每一週期產生合成圖
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # 儲存每一週期的output合成圖
    img = deprocess_image(x.copy())
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [26]:
!ls -l drive/Colab/images

total 475
-rw-r--r-- 1 root root 277368 Jul 17 23:37 input.jpg
-rw-r--r-- 1 root root 208857 Jul 17 23:56 startnight.jpg


In [29]:
!ls -l


total 688
drwxr-xr-x 1 root root   4096 Jul 17 23:33 datalab
drwxr-xr-x 2 root root   4096 Jul 17 23:34 drive
drwxr-xr-x 2 root root   4096 Jul 17 23:53 drive1
-rw-r--r-- 1 root root 345223 Jul 17 23:58 output_at_iteration_0.png
-rw-r--r-- 1 root root 336072 Jul 17 23:58 output_at_iteration_1.png


In [0]:
!cp out*.* drive/Colab/images